In [2]:
!pip3 install openai
!pip3 install gradio
!pip3 install tiktoken
!pip3 install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 184.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 263.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 225.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 206.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 224.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 213.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 246.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 313.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 567.8 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 125.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 140.2

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
import gradio as gr

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10000)

def get_response(input):
    print("------------")
    print(memory.load_memory_variables({}))
    print("------------")
    conversation_with_memory = ConversationChain(
        llm=llm, 
        memory=memory,
        verbose=False
    )
    return conversation_with_memory.predict(input=input)

def respond(message, chat_history):
        bot_message = get_response(message)
        chat_history.append((message, bot_message))
        return "", chat_history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) 
    msg = gr.Textbox(label="Prompt") 
    btn = gr.Button("Submit") 
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 
gr.close_all()
demo.launch()
    

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


------------
{'history': ''}
------------
------------
{'history': "Human: You are a security professional in a SaaS company and you are an Expert of Threat Modeling.\nAI: Yes, that's correct! As a security professional in a SaaS (Software as a Service) company, my main responsibility is to ensure the security of our software and protect our customers' data. Threat modeling is a crucial part of my job, as it helps identify potential vulnerabilities and threats in our system. By analyzing and understanding these threats, we can implement appropriate security measures to mitigate them."}
------------


In [14]:
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
import gradio as gr

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
prompt_template = "You are a security professional in a SaaS company and you are an Expert of Threat Modeling. You will do a threat modeling for {service_name} with these use cases. {usecases}"

llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

default_service_name = "ApplePay on Web Support"

default_usecases = """
Use Case 1: Zuora Tenants Integrate Adyen Apple Pay Flow
Tenants integrate Apple Pay Button on their website (Use other third-party services or implement by themselves)
End-user click the Apple Pay Button and go through the apple pay checkout flow
Tenants collect the end-user's apple pay paymentData token (Encrypted)
Tenants send the end-user's apple pay paymentData token to Zuora via /payment-methods REST API, see more on KC
After creating the Apple Pay payment method in Zuora, then Zuora can handle the subsequent transactions with Adyen Gateway

Use Case 2: Tenants Integrate Non-Adyen Apple Pay Flow
Tenants integrate Apple Pay Button on their website (Use other third-party services or implement by themselves)
End-user click the Apple Pay Button and go through the apple pay checkout flow
Tenants collect the end-user's apple pay paymentData token (Encrypted)
Tenants upload the Apple Pay certificate to Zuora
Tenants send the end-user's apple pay paymentData token to Zuora via /payment-methods/decryption REST API
After creating the Apple Pay payment method in Zuora, then Zuora can handle the subsequent transactions with Adyen Gateway
"""
def run_threat_modeling(service_name, usecases):
    return llm_chain.run(service_name=service_name, usecases=usecases) 


with gr.Blocks() as demo:
    service_name = gr.Textbox(label="Service Name", value=default_service_name)
    usecases = gr.Textbox(label="Use Cases", lines=10, value=default_usecases)
    btn = gr.Button("Run Threat Modeling")
    threat_modeling = gr.Textbox(label="Threat Modeling",lines=10)
    btn.click(run_threat_modeling, inputs=[service_name, usecases], outputs=[threat_modeling])
   
gr.close_all()
demo.launch()




Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [10]:
import gradio as gr

def generate_threat_model(service_name, use_cases):
    # Replace this with your actual threat modeling logic
    result = f"Threat model for service: {service_name}, use cases: {use_cases}"
    return result

iface = gr.Interface(
    fn=generate_threat_model, 
    inputs=[
        gr.inputs.Textbox(lines=1, placeholder="Enter Service Name Here..."), 
        gr.inputs.Textbox(lines=2, placeholder="Enter Use Cases Here...")
    ], 
    outputs="text",
    title="Threat Modeling",
    description="Enter the service name and use cases to generate a threat model."
)
gr.close_all()
iface.launch()

AttributeError: module 'gradio' has no attribute 'inputs'